In [1]:
### DAİRELERİN KONUMUNU VE AÇISINI DEĞİŞTİRMEK

In [4]:
import cv2
import numpy as np
import os
import random

def detect_white_regions(image):
    # Beyaz renk için bir eşik değeri belirleyin
    lower_white = np.array([200, 200, 200], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)

    # Belirlenen eşiklere göre maske oluşturun
    mask = cv2.inRange(image, lower_white, upper_white)

    # Maskeyi kullanarak beyaz bölgeleri tespit edin
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours, mask

def augment_images(input_folder, output_folder, num_images=1000, output_size=(800, 800), padding=15):
    try:
        os.makedirs(output_folder, exist_ok=True)
        
        # Giriş klasöründeki tüm dosyaları listele
        files = os.listdir(input_folder)
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]
        
        # İstenilen sayıda yeni görüntü oluştur
        for i in range(num_images):
            # Rastgele bir görüntü seç
            random_image_name = random.choice(image_files)
            image_path = os.path.join(input_folder, random_image_name)
            
            # Görüntüyü oku
            image = cv2.imread(image_path)
            if image is None:
                print(f"Görüntü okunamadı: {image_path}")
                continue
            
            # Beyaz bölgeleri bulma işlemi
            contours, mask = detect_white_regions(image)
            
            augmented_image = np.zeros((output_size[0], output_size[1], 3), dtype=np.uint8)
            
            # Orijinal görüntü boyutlarını al
            height, width = image.shape[:2]
            
            for contour in contours:
                # Konturu çevreleyen dikdörtgeni bulun ve tampon alan ekleyin
                x, y, w, h = cv2.boundingRect(contour)
                
                # Kare boyutunu belirleyin
                size = max(w, h)
                x = max(0, x - padding)
                y = max(0, y - padding)
                size = min(min(width - x, height - y), size + 2 * padding)
                
                # Kare içine beyaz bölgeyi yerleştirin
                square_region = image[y:y+size, x:x+size].copy()
                
                # Kare içinde beyaz bölgeyi rastgele bir açıyla döndürün
                angle = np.random.randint(0, 360)
                rotation_matrix = cv2.getRotationMatrix2D((size//2, size//2), angle, 1.0)
                rotated_square_region = cv2.warpAffine(square_region, rotation_matrix, (size, size))
                
                # Rotated bölgeyi rastgele bir konuma yerleştirin
                new_x = np.random.randint(0, output_size[1] - size)
                new_y = np.random.randint(0, output_size[0] - size)
                
                augmented_image[new_y:new_y+size, new_x:new_x+size] = rotated_square_region
                
                # Masked bölgeleri siyaha çevir
                image[y:y+size, x:x+size] = 0
            
            # Yeni dosya adı ve yolu oluşturun
            name, ext = os.path.splitext(random_image_name)
            new_image_name = f"{name}_donduruldu_{i+1}{ext}"
            new_image_path = os.path.join(output_folder, new_image_name)
            
            # Görüntüyü kaydedin
            if not cv2.imwrite(new_image_path, augmented_image):
                print(f"Görüntü kaydedilemedi: {new_image_path}")
            
            print(f"{i+1}/{num_images} images augmented.")
    
    except Exception as e:
        print(f"Hata: {e}")

# Kullanım örneği
if __name__ == "__main__":
    input_folder = "./giris"  # Giriş klasörü
    output_folder = "./cikis"  # Çıkış klasörü
    augment_images(input_folder, output_folder, num_images=3000)


1/3000 images augmented.
2/3000 images augmented.
3/3000 images augmented.
4/3000 images augmented.
5/3000 images augmented.
6/3000 images augmented.
7/3000 images augmented.
8/3000 images augmented.
9/3000 images augmented.
10/3000 images augmented.
11/3000 images augmented.
12/3000 images augmented.
13/3000 images augmented.
14/3000 images augmented.
15/3000 images augmented.
16/3000 images augmented.
17/3000 images augmented.
18/3000 images augmented.
19/3000 images augmented.
20/3000 images augmented.
21/3000 images augmented.
22/3000 images augmented.
23/3000 images augmented.
24/3000 images augmented.
25/3000 images augmented.
26/3000 images augmented.
27/3000 images augmented.
28/3000 images augmented.
29/3000 images augmented.
30/3000 images augmented.
31/3000 images augmented.
32/3000 images augmented.
33/3000 images augmented.
34/3000 images augmented.
35/3000 images augmented.
36/3000 images augmented.
37/3000 images augmented.
38/3000 images augmented.
39/3000 images augmen

KeyboardInterrupt: 

In [1]:
### DAİRELER YÖNÜ CE AÇISI ve uzaklığı DEĞİŞTİRİLİRKEN KOORDİNATLARININ KAYDEDİLMESİ

In [12]:
### DAİRELER YÖNÜ CE AÇISI ve uzaklığı DEĞİŞTİRİLİRKEN KOORDİNATLARININ KAYDEDİLMESİ
import cv2
import numpy as np
import os
import random

def detect_white_regions(image):
    # Beyaz renk için bir eşik değeri belirleyin
    lower_white = np.array([200, 200, 200], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)

    # Belirlenen eşiklere göre maske oluşturun
    mask = cv2.inRange(image, lower_white, upper_white)

    # Maskeyi kullanarak beyaz bölgeleri tespit edin
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours, mask

def resize_image(image, scale_factor):
    # Resmi rastgele bir ölçüde yeniden boyutlandırın
    height, width = image.shape[:2]
    new_height = int(height * scale_factor)
    new_width = int(width * scale_factor)
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

def augment_images(input_folder, output_folder, num_images=1000, output_size=(800, 800), padding=15):
    try:
        os.makedirs(output_folder, exist_ok=True)
        
        # Giriş klasöründeki tüm dosyaları listele
        files = os.listdir(input_folder)
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]
        
        # İstenilen sayıda yeni görüntü oluştur
        for i in range(num_images):
            # Rastgele bir görüntü seç
            random_image_name = random.choice(image_files)
            image_path = os.path.join(input_folder, random_image_name)
            
            # Görüntüyü oku
            image = cv2.imread(image_path)
            if image is None:
                print(f"Görüntü okunamadı: {image_path}")
                continue
            
            # Beyaz bölgeleri bulma işlemi
            contours, mask = detect_white_regions(image)
            
            augmented_image = np.zeros((output_size[0], output_size[1], 3), dtype=np.uint8)
            
            # Orijinal görüntü boyutlarını al
            height, width = image.shape[:2]
            
            for contour in contours:
                # Konturu çevreleyen dikdörtgeni bulun ve tampon alan ekleyin
                x, y, w, h = cv2.boundingRect(contour)
                
                # Kare boyutunu belirleyin
                size = max(w, h)
                x = max(0, x - padding)
                y = max(0, y - padding)
                size = min(min(width - x, height - y), size + 2 * padding)
                
                # Kare içine beyaz bölgeyi yerleştirin
                square_region = image[y:y+size, x:x+size].copy()
                
                # Rastgele bir ölçekte boyutlandırma faktörü seçin (0.5 ile 2 arasında)
                scale_factor = random.uniform(0.5, 2.0)
                resized_square_region = resize_image(square_region, scale_factor)
                
                # Kare içinde beyaz bölgeyi rastgele bir açıyla döndürün
                angle = np.random.randint(0, 360)
                rotation_matrix = cv2.getRotationMatrix2D((resized_square_region.shape[1]//2, resized_square_region.shape[0]//2), angle, 1.0)
                rotated_square_region = cv2.warpAffine(resized_square_region, rotation_matrix, (resized_square_region.shape[1], resized_square_region.shape[0]))
                
                # Rotated bölgeyi rastgele bir konuma yerleştirin
                new_x = np.random.randint(0, output_size[1] - resized_square_region.shape[1])
                new_y = np.random.randint(0, output_size[0] - resized_square_region.shape[0])
                
                augmented_image[new_y:new_y+rotated_square_region.shape[0], new_x:new_x+rotated_square_region.shape[1]] = rotated_square_region
                
                # Masked bölgeleri siyaha çevir
                image[y:y+size, x:x+size] = 0
            
            # Yeni dosya adı ve yolu oluşturun
            name, ext = os.path.splitext(random_image_name)
            new_image_name = f"{name}_donduruldu_{i+1}{ext}"
            new_image_path = os.path.join(output_folder, new_image_name)
            
            # Görüntüyü kaydedin
            if not cv2.imwrite(new_image_path, augmented_image):
                print(f"Görüntü kaydedilemedi: {new_image_path}")
            
            print(f"{i+1}/{num_images} images augmented.")
    
    except Exception as e:
        print(f"Hata: {e}")

# Kullanım örneği
if __name__ == "__main__":
    input_folder = "./giris"  # Giriş klasörü
    output_folder = "./cikis"  # Çıkış klasörü
    augment_images(input_folder, output_folder, num_images=100)


1/100 images augmented.
2/100 images augmented.
3/100 images augmented.
4/100 images augmented.
5/100 images augmented.
6/100 images augmented.
7/100 images augmented.
8/100 images augmented.
9/100 images augmented.
10/100 images augmented.
11/100 images augmented.
12/100 images augmented.
13/100 images augmented.
14/100 images augmented.
15/100 images augmented.
16/100 images augmented.
17/100 images augmented.
18/100 images augmented.
19/100 images augmented.
20/100 images augmented.
21/100 images augmented.
22/100 images augmented.
23/100 images augmented.
24/100 images augmented.
25/100 images augmented.
26/100 images augmented.
27/100 images augmented.
28/100 images augmented.
29/100 images augmented.
30/100 images augmented.
31/100 images augmented.
32/100 images augmented.
33/100 images augmented.
34/100 images augmented.
35/100 images augmented.
36/100 images augmented.
37/100 images augmented.
38/100 images augmented.
39/100 images augmented.
40/100 images augmented.
41/100 im

In [8]:
### XML DOSYASI OALUŞTURMA

In [14]:
### İŞLENMESİNİN İSTENEN VERİ SİYAH ZEMİN ÜZERİNE BEYAZ İSE BEYAZ OLAN CİSMİN TAMAMINI BİR KARE İÇERİSİNE ALARAK KARENİN KÖŞLERİNİ İŞARETLER 
### BU SAYEDE GÖRSELİN ETİKETLENMEK İSTENEN KISMI SEÇİLMİŞ OLUR 
### AMAÇ BASİT ETİKETLEME İŞLEMLERİ İÇİN İŞ YÜKÜNÜ AZALTMAK BİNLERCE VERİYİ TEK SEFERDE ETİKETLENMESİNİ SAĞLAMAK

import cv2
import numpy as np
import os
import random
import xml.etree.ElementTree as ET
from xml.dom import minidom

def detect_white_regions(image):
    # Beyaz renk için bir eşik değeri belirleyin
    lower_white = np.array([200, 200, 200], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)

    # Belirlenen eşiklere göre maske oluşturun
    mask = cv2.inRange(image, lower_white, upper_white)

    # Maskeyi kullanarak beyaz bölgeleri tespit edin
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours, mask

def resize_image(image, scale_factor):
    # Resmi rastgele bir ölçüde yeniden boyutlandırın
    height, width = image.shape[:2]
    new_height = int(height * scale_factor)
    new_width = int(width * scale_factor)
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

def create_xml_annotation(image_filename, objects, image_size):
    # XML dosyası için kök element oluşturun
    annotation = ET.Element("annotation")

    # Gerekli alt elementleri oluşturun
    folder = ET.SubElement(annotation, "folder")
    filename = ET.SubElement(annotation, "filename")
    path = ET.SubElement(annotation, "path")
    source = ET.SubElement(annotation, "source")
    database = ET.SubElement(source, "database")
    size = ET.SubElement(annotation, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")
    segmented = ET.SubElement(annotation, "segmented")

    # Alt elementlere değerleri atayın
    folder.text = ""
    filename.text = image_filename  # Resim dosya adı
    path.text = image_filename  # Resim dosya yolu
    database.text = "teknofest_atilay_tr"  # Veritabanı adı
    width.text = str(image_size[1])  # Görüntü genişliği
    height.text = str(image_size[0])  # Görüntü yüksekliği
    depth.text = str(image_size[2])  # Görüntü derinliği (kanal sayısı)
    segmented.text = "0"  # Segmentasyon yapılmamış

    # Her nesne için "object" elementleri oluşturun
    for obj in objects:
        object_elem = ET.SubElement(annotation, "object")
        name = ET.SubElement(object_elem, "name")
        pose = ET.SubElement(object_elem, "pose")
        truncated = ET.SubElement(object_elem, "truncated")
        difficult = ET.SubElement(object_elem, "difficult")
        occluded = ET.SubElement(object_elem, "occluded")
        bndbox = ET.SubElement(object_elem, "bndbox")
        xmin = ET.SubElement(bndbox, "xmin")
        xmax = ET.SubElement(bndbox, "xmax")
        ymin = ET.SubElement(bndbox, "ymin")
        ymax = ET.SubElement(bndbox, "ymax")

        # Her bir özellik için değerleri atayın
        name.text = "daire"  # Örneğin nesnenizin adı "daire" olarak varsayılmıştır
        pose.text = "Unspecified"
        truncated.text = "0"
        difficult.text = "0"
        occluded.text = "0"
        xmin.text = str(int(obj["cx"] - obj["w"] / 2))
        xmax.text = str(int(obj["cx"] + obj["w"] / 2))
        ymin.text = str(int(obj["cy"] - obj["h"] / 2))
        ymax.text = str(int(obj["cy"] + obj["h"] / 2))

    # XML dosyasını oluşturun ve kaydedin
    xml_filename = os.path.splitext(image_filename)[0] + ".xml"
    xml_path = os.path.join("./annotations", xml_filename)  # XML dosyasının kaydedileceği yol

    # XML verilerini minidom kullanarak bir string'e dönüştürün
    xml_string = minidom.parseString(ET.tostring(annotation, encoding='unicode')).toprettyxml(indent="  ")

    # Biçimlendirilmiş XML verilerini dosyaya yazın
    with open(xml_path, "w", encoding="utf-8") as f:
        f.write(xml_string)

def augment_images(input_folder, output_folder, num_images=1000, output_size=(800, 800), padding=15):
    try:
        os.makedirs(output_folder, exist_ok=True)
        os.makedirs("./annotations", exist_ok=True)  # XML dosyalarını kaydetmek için klasör oluştur

        # Giriş klasöründeki tüm dosyaları listele
        files = os.listdir(input_folder)
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]

        # İstenilen sayıda yeni görüntü oluştur
        for i in range(num_images):
            # Rastgele bir görüntü seç
            random_image_name = random.choice(image_files)
            image_path = os.path.join(input_folder, random_image_name)

            # Görüntüyü oku
            image = cv2.imread(image_path)
            if image is None:
                print(f"Görüntü okunamadı: {image_path}")
                continue

            # Beyaz bölgeleri bulma işlemi
            contours, mask = detect_white_regions(image)

            # Augmented görüntüyü oluştur
            augmented_image = np.zeros((output_size[0], output_size[1], 3), dtype=np.uint8)

            objects = []

            # Orijinal görüntü boyutlarını al
            height, width = image.shape[:2]

            for contour in contours:
                # Konturu çevreleyen dikdörtgeni bulun ve tampon alan ekleyin
                x, y, w, h = cv2.boundingRect(contour)

                # Kare boyutunu belirleyin
                size = max(w, h)
                x = max(0, x - padding)
                y = max(0, y - padding)
                size = min(min(width - x, height - y), size + 2 * padding)

                # Kare içine beyaz bölgeyi yerleştirin
                square_region = image[y:y+size, x:x+size].copy()

                # Rastgele bir ölçekte boyutlandırma faktörü seçin (0.5 ile 2 arasında)
                scale_factor = random.uniform(0.5, 2.0)
                resized_square_region = resize_image(square_region, scale_factor)

                # Kare içinde beyaz bölgeyi rastgele bir açıyla döndürün
                angle = np.random.randint(0, 360)
                rotation_matrix = cv2.getRotationMatrix2D((resized_square_region.shape[1]//2, resized_square_region.shape[0]//2), angle, 1.0)
                rotated_square_region = cv2.warpAffine(resized_square_region, rotation_matrix, (resized_square_region.shape[1], resized_square_region.shape[0]))

                # Rotated bölgeyi rastgele bir konuma yerleştirin
                new_x = np.random.randint(0, output_size[1] - rotated_square_region.shape[1])
                new_y = np.random.randint(0, output_size[0] - rotated_square_region.shape[0])

                # Rotated bölgenin merkezini hesaplayın
                center_x = new_x + rotated_square_region.shape[1] // 2
                center_y = new_y + rotated_square_region.shape[0] // 2

                # Koordinatları kaydedin (dairelerin merkezi, genişlik ve yükseklik)
                objects.append({
                    "cx": center_x, 
                    "cy": center_y, 
                    "w": rotated_square_region.shape[1], 
                    "h": rotated_square_region.shape[0]
                })

                # Yapıştırılan bölgeyi augment edilmiş görüntüye yerleştirin
                augmented_image[new_y:new_y+rotated_square_region.shape[0], new_x:new_x+rotated_square_region.shape[1]] = rotated_square_region

                # Masked bölgeleri siyaha çevir
                image[y:y+size, x:x+size] = 0

            # Yeni dosya adı ve yolu oluşturun
            name, ext = os.path.splitext(random_image_name)
            new_image_name = f"{name}_donduruldu_{i+1}{ext}"
            new_image_path = os.path.join(output_folder, new_image_name)

            # Görüntüyü kaydedin
            if not cv2.imwrite(new_image_path, augmented_image):
                print(f"Görüntü kaydedilemedi: {new_image_path}")

            # XML dosyasını oluşturun
            create_xml_annotation(new_image_name, objects, augmented_image.shape)

            print(f"{i+1}/{num_images} images augmented.")

    except Exception as e:
        print(f"Hata: {e}")

# Kullanım örneği
if __name__ == "__main__":
    input_folder = "./giris"  # Giriş klasörü
    output_folder = "./cikis"  # Çıkış klasörü
    augment_images(input_folder, output_folder, num_images=100)

1/100 images augmented.
2/100 images augmented.
3/100 images augmented.
4/100 images augmented.
5/100 images augmented.
6/100 images augmented.
7/100 images augmented.
8/100 images augmented.
9/100 images augmented.
10/100 images augmented.
11/100 images augmented.
12/100 images augmented.
13/100 images augmented.
14/100 images augmented.
15/100 images augmented.
16/100 images augmented.
17/100 images augmented.
18/100 images augmented.
19/100 images augmented.
20/100 images augmented.
21/100 images augmented.
22/100 images augmented.
23/100 images augmented.
24/100 images augmented.
25/100 images augmented.
26/100 images augmented.
27/100 images augmented.
28/100 images augmented.
29/100 images augmented.
30/100 images augmented.
31/100 images augmented.
32/100 images augmented.
33/100 images augmented.
34/100 images augmented.
35/100 images augmented.
36/100 images augmented.
37/100 images augmented.
38/100 images augmented.
39/100 images augmented.
40/100 images augmented.
41/100 im

In [11]:
import cv2
import numpy as np
import os
import random

def detect_white_regions(image):
    # Beyaz renk için bir eşik değeri belirleyin
    lower_white = np.array([200, 200, 200], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)

    # Belirlenen eşiklere göre maske oluşturun
    mask = cv2.inRange(image, lower_white, upper_white)

    # Maskeyi kullanarak beyaz bölgeleri tespit edin
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    return contours

def create_annotation_file(image_filename, objects):
    # Dosya adı ve yolu oluşturun
    annotation_filename = os.path.splitext(image_filename)[0] + ".txt"
    annotation_path = os.path.join("./annotations", annotation_filename)

    # Dosyayı açın
    with open(annotation_path, "w") as f:
        # Her nesne için koordinatları yazın
        for obj in objects:
            f.write(f"{obj['xmin']} {obj['xmax']} {obj['ymin']} {obj['ymax']}\n")

def augment_images(input_folder, output_folder, num_images=1000, output_size=(800, 800), padding=15):
    try:
        os.makedirs(output_folder, exist_ok=True)
        os.makedirs("./annotations", exist_ok=True)  # Annotasyon dosyalarını kaydetmek için klasör oluştur

        # Giriş klasöründeki tüm dosyaları listele
        files = os.listdir(input_folder)
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]

        # İstenilen sayıda yeni görüntü oluştur
        for i in range(num_images):
            # Rastgele bir görüntü seç
            random_image_name = random.choice(image_files)
            image_path = os.path.join(input_folder, random_image_name)

            # Görüntüyü oku
            image = cv2.imread(image_path)
            if image is None:
                print(f"Görüntü okunamadı: {image_path}")
                continue

            # Beyaz bölgeleri bulma işlemi
            contours = detect_white_regions(image)

            # Augmented görüntüyü oluştur
            augmented_image = np.zeros((output_size[0], output_size[1], 3), dtype=np.uint8)

            objects = []

            # Orijinal görüntü boyutlarını al
            height, width = image.shape[:2]

            for contour in contours:
                # Konturu çevreleyen dikdörtgeni bulun ve tampon alan ekleyin
                x, y, w, h = cv2.boundingRect(contour)

                # Kare boyutunu belirleyin
                size = max(w, h)
                # x ve y'yi sınırla
                x = max(0, min(x, width - size))
                y = max(0, min(y, height - size))
                size = min(min(width - x, height - y), size + 2 * padding)

                # Kare içine beyaz bölgeyi yerleştirin
                square_region = image[y:y+size, x:x+size].copy()

                # Kare içinde beyaz bölgeyi rastgele bir açıyla döndürün
                angle = np.random.randint(0, 360)
                rotation_matrix = cv2.getRotationMatrix2D((square_region.shape[1]//2, square_region.shape[0]//2), angle, 1.0)
                rotated_square_region = cv2.warpAffine(square_region, rotation_matrix, (square_region.shape[1], square_region.shape[0]))

                # Rotated bölgeyi rastgele bir konuma yerleştirin
                # new_x ve new_y değerlerini sınırla
                new_x = np.random.randint(0, output_size[1] - rotated_square_region.shape[1])
                new_y = np.random.randint(0, output_size[0] - rotated_square_region.shape[0])

                # Yapıştırılan alanın koordinatlarını kaydet
                objects.append({
                    "xmin": new_x,
                    "xmax": new_x + rotated_square_region.shape[1],
                    "ymin": new_y,
                    "ymax": new_y + rotated_square_region.shape[0]
                })

                # Yapıştırılan bölgeyi augment edilmiş görüntüye yerleştirin
                augmented_image[new_y:new_y+rotated_square_region.shape[0], new_x:new_x+rotated_square_region.shape[1]] = rotated_square_region

                # Masked bölgeleri siyaha çevir
                image[y:y+size, x:x+size] = 0

            # Yeni dosya adı ve yolu oluşturun
            name, ext = os.path.splitext(random_image_name)
            new_image_name = f"{name}_donduruldu_{i+1}{ext}"
            new_image_path = os.path.join(output_folder, new_image_name)

            # Görüntüyü kaydedin
            if not cv2.imwrite(new_image_path, augmented_image):
                print(f"Görüntü kaydedilemedi: {new_image_path}")

            # Annotasyon dosyasını oluşturun
            create_annotation_file(new_image_name, objects)

            print(f"{i+1}/{num_images} images augmented.")

    except Exception as e:
        print(f"Hata: {e}")

# Kullanım örneği
if __name__ == "__main__":
    input_folder = "./giris"  # Giriş klasörü
    output_folder = "./cikis"  # Çıkış klasörü
    augment_images(input_folder, output_folder, num_images=30)

1/30 images augmented.
2/30 images augmented.
3/30 images augmented.
4/30 images augmented.
5/30 images augmented.
6/30 images augmented.
7/30 images augmented.
8/30 images augmented.
9/30 images augmented.
10/30 images augmented.
11/30 images augmented.
12/30 images augmented.
13/30 images augmented.
14/30 images augmented.
15/30 images augmented.
16/30 images augmented.
17/30 images augmented.
18/30 images augmented.
19/30 images augmented.
20/30 images augmented.
21/30 images augmented.
22/30 images augmented.
23/30 images augmented.
24/30 images augmented.
25/30 images augmented.
26/30 images augmented.
27/30 images augmented.
28/30 images augmented.
29/30 images augmented.
30/30 images augmented.


In [ ]:
### ÇEMBERLERİ BOYAMAK İÇİN MAVİ YEŞİL VE KIRMIZI

In [3]:
import cv2
import numpy as np
import os
import random

def random_color(hue_range):
    # Rastgele bir renk tonu (Hue) seçin
    hue = random.randint(hue_range[0], hue_range[1])
    
    # Sabit doygunluk (Saturation) ve parlaklık (Value) değerleri belirleyin
    saturation = random.randint(100, 255)
    value = random.randint(100, 255)
    
    # HSV renk uzayında renk oluşturun
    hsv_color = np.uint8([[[hue, saturation, value]]])
    bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR)
    
    # Rastgele seçilen renk tonunu BGR formatında döndürün
    return tuple(bgr_color[0, 0])

def change_color(image):
    # Beyaz pikselleri tespit edin (belirli parlaklık aralığında olanlar seçilecek)
    lower_white = np.array([200, 200, 200], dtype=np.uint8)
    upper_white = np.array([255, 255, 255], dtype=np.uint8)
    white_mask = cv2.inRange(image, lower_white, upper_white)
    
    # Rastgele renk tonu seçin (mavi, yeşil veya kırmızı)
    color_choice = random.choice(['blue', 'green', 'red'])
    if color_choice == 'blue':
        new_color = random_color((100, 140))  # Mavi ton aralığı
    elif color_choice == 'green':
        new_color = random_color((35, 85))   # Yeşil ton aralığı
    else:  # 'red'
        new_color = random_color((0, 10))    # Kırmızı ton aralığı
    
    # Beyaz pikselleri rastgele seçilen renk ile doldurun
    image[white_mask > 0] = new_color
    
    return image

def process_images(input_folder, output_folder, num_images=1000):
    try:
        os.makedirs(output_folder, exist_ok=True)
        
        # Giriş klasöründeki tüm dosyaları listele
        files = os.listdir(input_folder)
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]
        
        # İstenilen sayıda yeni görüntü oluştur
        for i in range(num_images):
            # Rastgele bir görüntü seç
            random_image_name = random.choice(image_files)
            image_path = os.path.join(input_folder, random_image_name)
            
            # Görüntüyü oku (BGR formatında)
            image = cv2.imread(image_path)
            if image is None:
                print(f"Görüntü okunamadı: {image_path}")
                continue
            
            # Renk değiştirme işlemi
            modified_image = change_color(image.copy())  # Görüntüyü kopyalayarak değişiklik yapın
            
            # Yeni dosya adı ve yolu oluşturun
            new_image_name = f"modified_{i}_{random_image_name}"
            new_image_path = os.path.join(output_folder, new_image_name)
            
            # Görüntüyü kaydedin
            cv2.imwrite(new_image_path, modified_image)
            
            print(f"{i+1}/{num_images} images processed.")
    
    except Exception as e:
        print(f"Hata: {e}")

# Kullanım örneği
if __name__ == "__main__":
    input_folder = "./giris"  # Giriş klasörü
    output_folder = "./cikis_renkli"  # Çıkış klasörü
    process_images(input_folder, output_folder, num_images=100)


1/1000 images processed.
2/1000 images processed.
3/1000 images processed.
4/1000 images processed.
5/1000 images processed.
6/1000 images processed.
7/1000 images processed.
8/1000 images processed.
9/1000 images processed.
10/1000 images processed.
11/1000 images processed.
12/1000 images processed.
13/1000 images processed.
14/1000 images processed.
15/1000 images processed.
16/1000 images processed.
17/1000 images processed.
18/1000 images processed.
19/1000 images processed.
20/1000 images processed.
21/1000 images processed.
22/1000 images processed.
23/1000 images processed.
24/1000 images processed.
25/1000 images processed.
26/1000 images processed.
27/1000 images processed.
28/1000 images processed.
29/1000 images processed.
30/1000 images processed.
31/1000 images processed.
32/1000 images processed.
33/1000 images processed.
34/1000 images processed.
35/1000 images processed.
36/1000 images processed.
37/1000 images processed.
38/1000 images processed.
39/1000 images proces